In [1]:
import numpy as np 
import pandas as pd 
import bz2
import gc
import chardet
import re
import os

In [2]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm

from sklearn.model_selection import train_test_split, cross_val_score

import tflearn

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [27]:
data = pd.read_json("data/processed.json")
data = data[:10000]

In [28]:
len(data)

10000

In [40]:
train_x, test_x, train_y, test_y = train_test_split(data[0], data[1])

In [41]:
max_features = 10000
maxlen = 100
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_x.values)

In [42]:
tokenized_train = tokenizer.texts_to_sequences(train_x.values)
train_x = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

tokenized_test = tokenizer.texts_to_sequences(test_x.values)
test_x = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [43]:
net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy')

In [44]:
model = tflearn.DNN(net, tensorboard_verbose=1)
model.fit(train_x, train_y, validation_set=(test_x, test_y), show_metric=True, batch_size=32)

IndexError: list index out of range